In [1]:
%matplotlib qt
import matplotlib.pyplot as plt
import numpy as np
import pyfftw

from aotools.functions import circle
from numpy.fft import fftshift, ifftshift, fft2, ifft2
from scipy.signal import convolve

In [2]:
def rect(x, D=1.):
    x = np.abs(x)
    y = (x < 0.5*D).astype(np.float64)
    y[x == 0.5*D] = 0.5
    return y

def tri(t):
    t = np.abs(t)
    y = np.zeros(t.shape)
    idx = t < 1.0
    y[idx] = 1.0 - t[idx]
    return y

In [7]:
def fftwft2(data, delta):
    '''Forward FFT using pyfftw
    '''
    fftin = pyfftw.empty_aligned(data.shape, dtype='complex128')
    fftout = pyfftw.empty_aligned(data.shape, dtype='complex128')
    fftw = pyfftw.FFTW(fftin, fftout, axes=(0,1), direction='FFTW_FORWARD')
    fftin[:] = data +0.j
    fftw()
    return fftout * delta**2

def fftwift2(data, delta):
    '''Inverse FFT using pyfftw
    '''
    N = data.shape[0]
    fftin = pyfftw.empty_aligned(data.shape, dtype='complex128')
    fftout = pyfftw.empty_aligned(data.shape, dtype='complex128')
    fftw = pyfftw.FFTW(fftin, fftout, axes=(0,1), direction='FFTW_BACKWARD')
    fftin[:] = data + 0.j
    fftw()
    return fftout * (N * delta)**2

def npft2(data, delta):
    '''Forward FFT using numpy
    '''
    return fft2(data) * delta**2

def npift2(data, delta):
    '''Inverse FFT using numpy
    '''
    N = data.shape[0]
    return ifft2(data) * (N * delta)**2

def ft2(data, delta):
    '''Forward FFT using numpy + shifting (from Schmidt 2010)
    '''
    return fftshift(fft2(fftshift(data))) * delta**2

def ift2(data, delta):
    '''Inverse FFT using numpy + shifting (from Schmidt 2010)
    '''
    N = data.shape[0]
    return ifftshift(ifft2(ifftshift(data))) * (N * delta)**2

In [8]:
def myconv2(A, B, delta):
    '''FFT Convolution using Schmidt 2010 approach
    '''
    N = A.shape[0]
    return ift2(ft2(A, delta)*ft2(B, delta), 1/(N*delta))

def npconv2(A, B, delta):
    '''My own numpy FFT convolution
    '''
    N = A.shape[0]
    return npift2(npft2(A, delta)*npft2(B, delta), 1/(N*delta))

def fftwconv2(A, B, delta):
    '''My own FFT convolution using pyfftw
    '''
    N = A.shape[0]
    return fftwift2(fftwft2(A, delta)*fftwft2(B, delta), 1/(N*delta))

In [9]:
# 16 meter square grid, 256 pixels, with a 2 meter wide square
N, L, w = 256, 16, 2
delta = L/N
x = np.arange(-N/2, N/2) * delta
x, y = np.meshgrid(x, x)

# Make a square (2D Rect Function)
A = rect(x/w) * rect(y/w)
B = rect(x/w) * rect(y/w)
# Analytic solution to the convolution
C_cont = w**2 * tri(x/w) * tri(y/w)

In [14]:
C0 = myconv2(A, B, delta)
C1 = convolve(A, B, 'same', 'direct') * delta**2
C2 = fftwconv2(A, B, delta)
f, axes = plt.subplots(2,2)
axes[0,0].set_title("C2, pyfftw convolution")
im0 = axes[0,0].imshow(fftshift(C2).real, origin='lower', cmap='plasma')
f.
axes[0,1].set_title("C_cont, analytic solution")
axes[0,1].imshow(C_cont, origin='lower', cmap='plasma')
axes[1,0].set_title("C2 - C_cont")
axes[1,0].imshow(fftshift(C2).real - C_cont, origin='lower', cmap='plasma')
axes[1,1].set_title("C1 - C_cont")
axes[1,1].imshow(C1 - C_cont, origin='lower', cmap='plasma')